# 대형아울렛 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
import matplotlib
from numpy import inf

matplotlib.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'Malgun Gothic'
warnings.filterwarnings('ignore')
plt.style.use('seaborn')
%matplotlib inline

## 서울
http://data.seoul.go.kr/dataList/10128/S/2/datasetView.do
- chrome 말고 internet explorer로 들어가야 오류가 나지 않습니다

In [2]:
# 대형마트, 백화점, 전문점, 쇼핑센터, 복합쇼핑몰, 그밖의 대규모점포의 갯수에 대한 데이터
seoul = pd.read_table("../data/external_data/raw_data/서울시 유통업체현황 (동별) 통계.txt", 
                      usecols=[1, 2, 6, 9, 15, 18], 
                      skiprows=[0, 2, 3])

In [3]:
# 중구 / 노원구만
seoul = seoul.query('자치구 == "중구" or 자치구=="노원구"').query('동 != "소계"').reset_index(drop=True)

# '-'값을 0으로 대체합니다
seoul = seoul.replace('-', 0)

# 수치형으로
for column in seoul.columns[2:]:
    seoul[column] = seoul[column].map(int)
    
# 대형아울렛 입지유형은 대형마트/백화점/쇼핑센터/복합쇼핑몰 존재 유무에 따라 나누도록 하겠습니다
seoul['합계'] = seoul[['대형마트', '백화점', '쇼핑센터', '복합쇼핑몰']].sum(axis=1)

# 대구데이터와 합칠 예정이므로 시도 정보를 붙여줍니다
seoul['시도'] = "서울특별시"

In [4]:
seoul = seoul[['시도', '자치구', '동', '대형마트', '백화점', '쇼핑센터', '복합쇼핑몰', '합계']]

In [5]:
seoul.head()

,시도,자치구,동,대형마트,백화점,쇼핑센터,복합쇼핑몰,합계
0,서울특별시,중구,소공동,0,2,0,0,2
1,서울특별시,중구,회현동,1,1,2,0,4
2,서울특별시,중구,명동,0,0,1,0,1
3,서울특별시,중구,광희동,0,0,4,0,4
4,서울특별시,중구,을지로동,0,0,0,0,0


## 대구
- 수성구 : http://kosis.kr/statHtml/statHtml.do?orgId=556&tblId=DT_55601_H001001
- 중구 : http://kosis.kr/statHtml/statHtml.do?orgId=551&tblId=DT_55101_H001001

In [6]:
suseong = pd.read_csv("../data/external_data/raw_data/유통업체_현황_수성구.csv", encoding='cp949', 
                      usecols=[0, 4, 10, 13], 
                      skiprows=[0, 2, 3, 4, 5])

junggu = pd.read_csv("../data/external_data/raw_data/유통업체_현황_중구.csv", encoding='cp949', 
                     usecols=[0, 4, 10, 13, 16], 
                     skiprows=[0, 2, 3, 4, 5])

In [7]:
suseong['복합쇼핑몰'] = 0

suseong['자치구'] = "수성구"; suseong['시도'] = "대구광역시"
junggu['자치구'] = "중구"; junggu['시도'] = "대구광역시"

suseong.rename(columns={"행정구역별(1)":"동"}, inplace=True)
junggu.rename(columns={"동별(1)":"동"}, inplace=True)

In [8]:
daegu = pd.concat([suseong, junggu], axis=0)[['시도', '자치구', '동', '대형마트(할인점)', '백화점', '쇼핑센터', '복합쇼핑몰']]

In [9]:
# '-'값을 0으로 대체합니다
daegu = daegu.replace('-', 0)

# 수치형으로 변경
for column in daegu.columns[3:]:
    daegu[column] = daegu[column].map(int)

# 대형아울렛 입지유형은 대형마트/백화점/쇼핑센터/복합쇼핑몰 존재 유무에 따라 나누도록 하겠습니다
daegu['합계'] = daegu[['대형마트(할인점)', '백화점', '쇼핑센터', '복합쇼핑몰']].sum(axis=1)

# 서울데이터와 합치므로 컬럼 이름을 일치시켜 줍니다
daegu.rename(columns={"대형마트(할인점)":"대형마트"}, inplace=True)

In [10]:
daegu.head()

,시도,자치구,동,대형마트,백화점,쇼핑센터,복합쇼핑몰,합계
0,대구광역시,수성구,범어1동,0,0,0,0,0
1,대구광역시,수성구,범어2동,0,0,0,0,0
2,대구광역시,수성구,범어3동,0,0,0,0,0
3,대구광역시,수성구,범어4동,0,0,0,0,0
4,대구광역시,수성구,만촌1동,1,0,0,0,1


## 최종

In [11]:
seoul_daegu = pd.concat([seoul, daegu], axis=0).reset_index(drop=True)

In [12]:
seoul_daegu["outlet_region"] = seoul_daegu['합계'].apply(lambda x:x>=1)

In [13]:
# 서울, 대구의 아울렛 지역
seoul_daegu.query('outlet_region == True')[['시도', '자치구', '동']]

,시도,자치구,동
0,서울특별시,중구,소공동
1,서울특별시,중구,회현동
2,서울특별시,중구,명동
3,서울특별시,중구,광희동
5,서울특별시,중구,황학동
6,서울특별시,중구,신당동
8,서울특별시,노원구,월계3동
11,서울특별시,노원구,중계1동
13,서울특별시,노원구,상계2동
15,서울특별시,노원구,중계2.3동
